In [1]:
#!pip install datasets transformers==4.39.2
#!pip install sentencepiece
#!pip install accelerate -U
os.environ["WANDB_DISABLED"] = "true"

In [38]:
from datasets import load_dataset, load_metric, concatenate_datasets
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import DataLoader

In [2]:
torch.cuda.get_device_name()

'Tesla P100-PCIE-12GB'

In [3]:
device = torch.device('cuda')

# Feature Extraction

In [4]:
train_data = load_dataset('wikisql', split='train')
val_data = load_dataset('wikisql', split='validation')
test_data = load_dataset('wikisql', split='test')

In [5]:
train_data[0]

{'phase': 1,
 'question': 'Tell me what the notes are for South Australia ',
 'table': {'header': ['State/territory',
   'Text/background colour',
   'Format',
   'Current slogan',
   'Current series',
   'Notes'],
  'page_title': '',
  'page_id': '',
  'types': ['text', 'text', 'text', 'text', 'text', 'text'],
  'id': '1-1000181-1',
  'section_title': '',
  'caption': '',
  'rows': [['Australian Capital Territory',
    'blue/white',
    'Yaa·nna',
    'ACT · CELEBRATION OF A CENTURY 2013',
    'YIL·00A',
    'Slogan screenprinted on plate'],
   ['New South Wales',
    'black/yellow',
    'aa·nn·aa',
    'NEW SOUTH WALES',
    'BX·99·HI',
    'No slogan on current series'],
   ['New South Wales',
    'black/white',
    'aaa·nna',
    'NSW',
    'CPX·12A',
    'Optional white slimline series'],
   ['Northern Territory',
    'ochre/white',
    'Ca·nn·aa',
    'NT · OUTBACK AUSTRALIA',
    'CB·06·ZZ',
    'New series began in June 2011'],
   ['Queensland',
    'maroon/white',
    'nnn·aaa

In [6]:
START_TOK = '[SOS] '
def format_dataset(example):
    return {'input': START_TOK+example['question'], 'target': example['sql']['human_readable']}


In [7]:
train_data = train_data.map(format_dataset, remove_columns=train_data.column_names)
val_data = val_data.map(format_dataset, remove_columns=val_data.column_names)
test_data = test_data.map(format_dataset, remove_columns=test_data.column_names)

train_data[0]


{'input': '[SOS] Tell me what the notes are for South Australia ',
 'target': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'}

# Tokenization

In [8]:
CHECKPOINT = 'google-t5/t5-small'
tokenizer = T5Tokenizer.from_pretrained(CHECKPOINT)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Finding appropriate Max_Length

In [9]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
    x["input_len"] = len(tokenizer(x["input"]).input_ids)
    x["input_longer_128"] = int(x["input_len"] > 128)
    x["input_longer_64"] = int(x["input_len"] > 64)
    x["input_longer_32"] = int(x["input_len"] > 32)

    x["out_len"] = len(tokenizer(x["target"]).input_ids)
    x["out_longer_128"] = int(x["out_len"] > 128)
    x["out_longer_64"] = int(x["out_len"] > 64)
    x["out_longer_32"] = int(x["out_len"] > 32)
    return x


In [10]:
train_stats = train_data.map(map_to_length, num_proc=4)

In [11]:
val_stats = val_data.map(map_to_length, num_proc=4)

In [12]:
test_stats = test_data.map(map_to_length, num_proc=4)

In [13]:
all_merged = concatenate_datasets([train_stats,
                                   val_stats,
                                  test_stats])

##### Some Analysis on lengths

In [14]:
def compute_and_print_stats(x, sample_size):
    if len(x["input_len"]) == sample_size:
        print(
            "Input Max: {}, Input Mean: {:.5f}, Input>32:{},  Input>128:{:.5f}, Input>64:{:.5f} \nOutput Max: {}, Output Mean:{:.5f}, Output>32:{}, Output>128:{:.5f}, Output>64:{:.5f}".format(
                max(x["input_len"]),
                sum(x["input_len"]) / sample_size,
                sum(x["input_longer_32"]) / sample_size,
                sum(x["input_longer_128"]) / sample_size,
                sum(x["input_longer_64"]) / sample_size,
                max(x["out_len"]),
                sum(x["out_len"]) / sample_size,
                sum(x["out_longer_32"]) / sample_size,
                sum(x["out_longer_128"]) / sample_size,
                sum(x["out_longer_64"]) / sample_size,
            )
        )

In [15]:
# All Data
output = all_merged.map(
  lambda x: compute_and_print_stats(x, all_merged.shape[0]), 
  batched=True,
  batch_size=-1,
)

Map:   0%|          | 0/80654 [00:00<?, ? examples/s]

Input Max: 94, Input Mean: 21.73463, Input>32:0.07684677759317579,  Input>128:0.00000, Input>64:0.00046 
Output Max: 176, Output Mean:21.57647, Output>32:0.05963746373397476, Output>128:0.00002, Output>64:0.00035


In [16]:
# Train Data
output = train_stats.map(
  lambda x: compute_and_print_stats(x, train_stats.shape[0]), 
  batched=True,
  batch_size=-1,
)

Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

Input Max: 94, Input Mean: 21.71997, Input>32:0.07614231212847129,  Input>128:0.00000, Input>64:0.00043 
Output Max: 176, Output Mean:21.57257, Output>32:0.05971076213290746, Output>128:0.00004, Output>64:0.00032


In [17]:
# Val Data
output = val_stats.map(
  lambda x: compute_and_print_stats(x, val_stats.shape[0]), 
  batched=True,
  batch_size=-1,
)

Map:   0%|          | 0/8421 [00:00<?, ? examples/s]

Input Max: 83, Input Mean: 21.78126, Input>32:0.07552547203420021,  Input>128:0.00000, Input>64:0.00071 
Output Max: 79, Output Mean:21.45707, Output>32:0.05640660254126588, Output>128:0.00000, Output>64:0.00059


### Tokenizing and Padding

In [18]:
BUFFER = 2 # start end tokens
MAX_LENGTH = 64 + BUFFER

In [19]:
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], padding='max_length', max_length=MAX_LENGTH, truncation=True)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], padding='max_length', max_length=MAX_LENGTH, truncation=True)
    
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }


    return encodings

In [20]:
finaltrain_data = train_data.map(convert_to_features, batched=True, remove_columns=train_data.column_names, num_proc=4)
finalval_data = val_data.map(convert_to_features, batched=True, remove_columns=val_data.column_names, num_proc=4)
#finaltest_data = test_data.map(convert_to_features, batched=True, remove_columns=test_data.column_names, num_proc=4)

In [21]:
columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

In [22]:
finaltrain_data.set_format(type='torch', columns=columns, device=device)
finalval_data.set_format(type='torch', columns=columns, device=device)
#finaltest_data.set_format(type='torch', columns=columns, device=device)

In [23]:
finaltrain_data[1]['input_ids'], finaltrain_data[0]['input_ids'].shape

(tensor([ 784,  134, 3638,  908,  363,   19,    8,  750,  939,  213,    8,  126,
          939, 1553,   16, 1515, 2722,   58,    1,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0], device='cuda:0'),
 torch.Size([66]))

In [24]:
tokenizer.decode(finaltrain_data[0]['input_ids'])

'[SOS] Tell me what the notes are for South Australia</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Training

In [25]:
model_dir = f"./t5-checkpoints/"

In [27]:
args = Seq2SeqTrainingArguments(model_dir,
                               dataloader_pin_memory=False)

In [28]:
model = T5ForConditionalGeneration.from_pretrained(CHECKPOINT, device_map=device)

In [29]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=finaltrain_data,
    eval_dataset=finalval_data,
)

/home/singhal.n/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [30]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: namanrocks1999 (neu_nmnsnghl). Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.777000
1000,0.308100
1500,0.264500
2000,0.248000
2500,0.232000
3000,0.217600
3500,0.210000
4000,0.213400
4500,0.200700
5000,0.194300


TrainOutput(global_step=21135, training_loss=0.19133752615761468, metrics={'train_runtime': 2590.2534, 'train_samples_per_second': 65.27, 'train_steps_per_second': 8.159, 'total_flos': 2949576308490240.0, 'train_loss': 0.19133752615761468, 'epoch': 3.0})

In [32]:
trainer.save_model('./t5-model')

## Generating SQL

In [60]:
def translate_to_sql(local_model, text):
    inputs = tokenizer(text, padding='longest', max_length=MAX_LENGTH, truncation=True, return_tensors='pt')
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    output = local_model.generate(input_ids, attention_mask=attention_mask, max_length=64)


    return tokenizer.decode(output[0], skip_special_tokens=True)

def generate_sql_on_test(data, local_model):
    length = data.shape[0]
    query = data['input']
    expected = data['target']
        
    for i in range(length):
        print(f"QUERY - {query[i]}")
        translated = translate_to_sql(local_model, query[i])
        print(f"Prediction - {translated}")
        print(f"Expected = {expected[i]}")
        print("="*50)

In [61]:
generate_sql_on_test(test_data.select(range(10)), model.to("cpu"))

QUERY - [SOS] What is terrence ross' nationality
Prediction - SELECT Nationality FROM table WHERE Name = terrence ross
Expected = SELECT Nationality FROM table WHERE Player = Terrence Ross
QUERY - [SOS] What clu was in toronto 1995-96
Prediction - SELECT clu FROM table WHERE Location = toronto 1995-96
Expected = SELECT School/Club Team FROM table WHERE Years in Toronto = 1995-96
QUERY - [SOS] which club was in toronto 2003-06
Prediction - SELECT Club FROM table WHERE Location = toronto 2003-06
Expected = SELECT School/Club Team FROM table WHERE Years in Toronto = 2003-06
QUERY - [SOS] how many schools or teams had jalen rose
Prediction - SELECT COUNT Schools/Teams FROM table WHERE Player = Jalen Rose
Expected = SELECT COUNT School/Club Team FROM table WHERE Player = Jalen Rose
QUERY - [SOS] Where was Assen held?
Prediction - SELECT Venue FROM table WHERE Team = assen
Expected = SELECT Round FROM table WHERE Circuit = Assen
QUERY - [SOS] What was the number of race that Kevin Curtain wo

wandb: Waiting for W&B process to finish... (success).
